Set up Python libraries

In [ ]:
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

Set up billing project and path variables

In [ ]:
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

GS_RELEASE_PATH = 'gs://amp-pd-data/releases/2019_v1release_1015'
GS_CLINICAL_RELEASE_PATH = f'{GS_RELEASE_PATH}/clinical'

GS_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2019_v1release_1015/wgs'
GS_WGS_RELEASE_PLINK_PATH = os.path.join(GS_WGS_RELEASE_PATH, 'plink')
GS_WGS_RELEASE_GATK_PATH = os.path.join(GS_WGS_RELEASE_PATH, 'gatk')

BQ_RELEASE_DATASET = 'amp-pd-research.2019_v1release_1015'


print(BILLING_PROJECT_ID)
print(GS_CLINICAL_RELEASE_PATH)
print(GS_WGS_RELEASE_PLINK_PATH)
print(GS_WGS_RELEASE_GATK_PATH)

Set up functions

In [ ]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}')
    !$command

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))
    
# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)
    
# Get the data from a query
def bq_query(query):
    """Return the contents of a query against BigQuery"""
    return pd.read_gbq(
        query,
        project_id=BILLING_PROJECT_ID,
        dialect='standard')

Set up folders

In [ ]:
%%bash

mkdir -p ~/bin/data_temp
cd ~/bin/
ls

Import PD GWAS summary statistics

In [ ]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp {WORKSPACE_BUCKET}/nallsEtAl2019_excluding23andMe_allVariants.tab.zip ~/bin/data_temp/')

Check sumstats file is in the /bin/data_temp directory

In [ ]:
%%bash

ls ~/bin/data_temp

Unzip sumstats file

In [ ]:
%%bash

cd ~/bin/data_temp

unzip nallsEtAl2019_excluding23andMe_allVariants.tab.zip

In [ ]:
%%bash

ls ~/bin/data_temp

In [ ]:
%%bash

head ~/bin/data_temp/nallsEtAl2019_excluding23andMe_allVariants.tab

In [ ]:
%%bash

ls ~/bin/data_temp/

In [ ]:
%%bash
head ~/bin/data_temp/PDGWAS_for_FUMA_MT.txt

In [ ]:
%%bash
cd ~/bin/data_temp/
zip PDGWAS_for_FUMA_MT.txt.zip PDGWAS_for_FUMA_MT.txt

In [ ]:
%%bash
ls ~/bin/data_temp/

Write FUMA formatted file to workspace bucket

In [ ]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r ~/bin/data_temp/PDGWAS_for_FUMA_MT.txt.zip {WORKSPACE_BUCKET}')